# Demo object preparation

In [1]:
import dandelion as ddl
import scanpy as sc
import warnings

warnings.filterwarnings('ignore')

This is how we created the objects that you downloaded as part of the quickstart tutorial. The syntax may be of use to you when importing your own data.

We started off by downloading public 10X data:

```
mkdir dandelion_tutorial
mkdir -p dandelion_tutorial/vdj_nextgem_hs_pbmc3
mkdir -p dandelion_tutorial/vdj_v1_hs_pbmc3
mkdir -p dandelion_tutorial/sc5p_v2_hs_PBMC_10k
mkdir -p dandelion_tutorial/sc5p_v2_hs_PBMC_1k
cd dandelion_tutorial/vdj_v1_hs_pbmc3;
wget -O filtered_feature_bc_matrix.h5 https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_v1_hs_pbmc3/vdj_v1_hs_pbmc3_filtered_feature_bc_matrix.h5;
wget -O filtered_contig_annotations.csv https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_v1_hs_pbmc3/vdj_v1_hs_pbmc3_b_filtered_contig_annotations.csv;
wget -O filtered_contig.fasta https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_v1_hs_pbmc3/vdj_v1_hs_pbmc3_b_filtered_contig.fasta;
cd ../vdj_nextgem_hs_pbmc3
wget -O filtered_feature_bc_matrix.h5 https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_nextgem_hs_pbmc3/vdj_nextgem_hs_pbmc3_filtered_feature_bc_matrix.h5;
wget -O filtered_contig_annotations.csv https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_nextgem_hs_pbmc3/vdj_nextgem_hs_pbmc3_b_filtered_contig_annotations.csv;
wget -O filtered_contig.fasta https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_nextgem_hs_pbmc3/vdj_nextgem_hs_pbmc3_b_filtered_contig.fasta;
cd ../sc5p_v2_hs_PBMC_10k;
wget -O filtered_feature_bc_matrix.h5 https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_10k/sc5p_v2_hs_PBMC_10k_filtered_feature_bc_matrix.h5;
wget -O filtered_contig_annotations.csv https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_10k/sc5p_v2_hs_PBMC_10k_b_filtered_contig_annotations.csv;
wget -O filtered_contig.fasta https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_10k/sc5p_v2_hs_PBMC_10k_b_filtered_contig.fasta;
cd ../sc5p_v2_hs_PBMC_1k;
wget -O filtered_feature_bc_matrix.h5 wget https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_1k/sc5p_v2_hs_PBMC_1k_filtered_feature_bc_matrix.h5;
wget -O filtered_contig_annotations.csv wget https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_1k/sc5p_v2_hs_PBMC_1k_b_filtered_contig_annotations.csv;
wget -O filtered_contig.fasta https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_1k/sc5p_v2_hs_PBMC_1k_b_filtered_contig.fasta;
```

Once the data was downloaded, we returned to the `dandelion_tutorial` directory and created a `meta.csv` file with the following contents:

```
sample,prefix,individual
vdj_v1_hs_pbmc3,vdj_v1_hs_pbmc3,vdj_v1_hs_pbmc3
vdj_nextgem_hs_pbmc3,vdj_nextgem_hs_pbmc3,vdj_nextgem_hs_pbmc3
sc5p_v2_hs_PBMC_10k,sc5p_v2_hs_PBMC_10k,sc5p_v2_hs_PBMC_10k
sc5p_v2_hs_PBMC_1k,sc5p_v2_hs_PBMC_1k,sc5p_v2_hs_PBMC_1k
```

We then ran the preprocessing pipeline with the aid of the Dandelion singularity container:

```
singularity pull library://kt16/default/sc-dandelion:latest
singularity run -B $PWD sc-dandelion_latest.sif dandelion-preprocess --meta meta.csv --file_prefix filtered
```

At this stage, the data was ready for import. Create a list of samples to use.

In [2]:
samples = ['sc5p_v2_hs_PBMC_1k', 'sc5p_v2_hs_PBMC_10k', 'vdj_v1_hs_pbmc3', 'vdj_nextgem_hs_pbmc3']

Import the GEX data and combine it into a single object.

In [3]:
adata_list = []
for sample in samples:
    adata = sc.read_10x_h5('dandelion_tutorial/' + sample +'/filtered_feature_bc_matrix.h5', gex_only=True)
    adata.obs['sampleid'] = sample
    # rename cells to sample id + barcode, and cleave the trailing -#
    adata.obs_names = [str(sample)+'_'+str(j).split('-')[0] for j in adata.obs_names]
    adata.var_names_make_unique()
    adata_list.append(adata)
#no need for index_unique as we already made barcodes unique by prepending the sample ID
adata = adata_list[0].concatenate(adata_list[1:], index_unique=None)

Import the Dandelion preprocessing output, and then combine that into a matching single object as well.

In [4]:
vdj_list = []
for sample in samples:
    vdj = ddl.read_10x_airr('dandelion_tutorial/'+sample+'/dandelion/filtered_contig_dandelion.tsv')
    #the dandelion output already has the sample ID prepended at the start of each contig
    vdj_list.append(vdj)
vdj = ddl.concat(vdj_list)

Do standard GEX processing via Scanpy.

In [5]:
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var.highly_variable]
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_pcs=20)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5)

And that's it! The objects were saved and put up on FTP for you to download.

In [6]:
adata.write("demo-gex.h5ad")
vdj.write('demo-vdj.h5ddl')